In [1]:
!pip install lyft-dataset-sdk

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 102kB 4.5MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=721068a73c4e300eaca5d7a6676bea4abdfa9f89830412b14d32c3843452b66a
  Stored in directory: /tmp/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for pathspec: filename=pathspec-0.6.0-cp36-none-any.whl size=26671 sha256=fdfbbe390eb6cf0773b4293933c7d4023c89a29d705ae11010f039268b4859f9
  Stored in directory: /tmp/.cache/pip/wheels/62/b8/e1/e2719465b5947c40cd85d613d6cb33449b86a1ca5a6c574269
Successfully built fire pathspec


In [2]:
!pip install squaternion

     |████████████████████████████████| 61kB 3.1MB/s 


In [3]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np
# Load the SDK
from lyft_dataset_sdk.lyftdataset import LyftDataset, LyftDatasetExplorer
from lyft_dataset_sdk.utils.data_classes import Box, LidarPointCloud, RadarPointCloud 
from lyft_dataset_sdk.utils.geometry_utils import BoxVisibility, box_in_image, view_points
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision, get_class_names, get_average_precisions
import math
# from source.utilities import print_progress
from pyquaternion import Quaternion
import os
import time
from multiprocessing import Process
from squaternion import euler2quat, quat2euler
from tqdm import tqdm_notebook as tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [4]:
# gotta do this for LyftDataset SDK, it expects folders to be named as `images`, `maps`, `lidar`

!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_images images
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_maps maps
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_lidar lidar
!ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_data data

In [5]:
level5data = LyftDataset(data_path='.', json_path='data/', verbose=True)

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 17.6 seconds.
Reverse indexing ...
Done reverse indexing in 7.0 seconds.


In [6]:
train = pd.read_csv('../input/lyft3d-inference-kernel-train-dataset/lyft3d_pred_train.csv')
train_num = len(train)
print(train_num)

22680


In [7]:
train.head()

,Id,PredictionString
0,e7475eae27620f8590b3452bd40adbfa8d18c19ab6ace1...,1.0 2403.807026839478 883.3258624373942 -17.96...
1,aeb380eb301cb0bf695241d7773b9ad8877b6a5dabe800...,1.0 1719.3651381858178 1280.4062232617846 -18....
2,848b9e783ac3eb6ddd873a5e34f618e85f7f45829a09fe...,0.9921568627450981 1118.1755661004474 1714.547...
3,8e5b719f1f69bd1807ad16833768d56bfde90015d79e47...,0.9254901960784314 2560.581448526204 706.33268...
4,fc5cd309c4e7248439e6eef7142abe37fbf08e9a35be1a...,0.996078431372549 837.379520145173 2519.156064...


In [8]:
pred_animal = pd.read_csv('../input/animal-evaluation/lyft3d_train_pred_animal.csv')
pred_animal.head()

,Id,PredictionString
0,088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8...,0.9969266057014465 1088.9559058161115 1650.995...
1,a371e1aada87530e7b8bf05414b8ec570d2304af95fa7a...,0.9967356324195862 2209.7699097225754 1000.409...
2,55db93f840b21f34da44deeb00d14df4609c727d1aefbc...,0.9941755533218384 1086.9383649477202 1651.673...
3,dbe222818fd1b10d0317e8b86bc8b1fa3ffd82b704cd52...,0.9490238428115845 2259.030045702017 975.75998...
4,4c462942f62aa048e5cc8c2a9480d426a293cac49c1c17...,0.9964942336082458 1090.0592552875721 1650.932...


In [9]:
pred_motorcycle = pd.read_csv('../input/motorcycle-train-evaluation/lyft3d_train_pred_motorcycle.csv')
pred_motorcycle.head()

,Id,PredictionString
0,039b87bf695195c1951fe3eac936e510fb818ddf632c65...,0.9916443228721619 1020.8500659277034 1736.386...
1,a2f2885ff1b08e3ab43d7b4b77b1599dd2161b0239ecfb...,0.9582344889640808 1118.2236872052822 1661.102...
2,d9a52c3e47de6edd38159d24e3aa3cb48288e6a5528583...,0.9827399253845215 1021.0009336005861 1735.837...
3,b6e45654151244120089af3b756fa632244712a1e33a58...,0.9985979199409485 2225.18721434149 982.947737...
4,68a7e4217a06bcff33f9ee791f585c639baa883b1a84f4...,0.9863404035568237 2074.5482804343383 1088.134...


In [10]:
def get_boxes_list_3d(pred_boxes):
    if not pd.isna(pred_boxes):
        pred_boxes = pred_boxes.split(' ')
        pred_boxes = pred_boxes[:-1]
        pred_boxes_list = np.reshape(pred_boxes, (len(pred_boxes)//9, 9))
    else:
        pred_boxes_list = []
    return pred_boxes_list

In [11]:
sample_tokens = []
for idx in range(train_num):
    sample_tokens.append(pred_animal.iloc[idx]['Id'])

In [12]:
columns = ['Id', 'PredictionString']
df_pred = pd.DataFrame(columns=columns)   

In [13]:
for i, sample_token in tqdm(enumerate(sample_tokens)):
    row = [] 
    result = ''
    token = sample_token
    row.append(token)
    df = train.loc[train['Id'] == token]
    pred_str = str(np.array(df['PredictionString'])[0])
    pred_str_animal = pred_animal.iloc[i]['PredictionString']
    df = pred_motorcycle.loc[pred_motorcycle['Id'] == token]
    pred_str_motorcycle = str(np.array(df['PredictionString'])[0])
    if pd.isna(pred_str_animal):
        pred_str_animal = ''
    if pd.isna(pred_str_motorcycle):
        pred_str_motorcycle = ''   
    result = pred_str + pred_str_animal + pred_str_motorcycle
    row.append(result)  
    df_row = pd.DataFrame([row], columns=columns)
    df_pred = df_pred.append(df_row)

In [14]:
df_pred.head()

,Id,PredictionString
0,088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8...,1.0 1111.7313500943123 1710.079884903943 -23.6...
0,a371e1aada87530e7b8bf05414b8ec570d2304af95fa7a...,1.0 2209.1951943101826 966.8429535510375 -18.5...
0,55db93f840b21f34da44deeb00d14df4609c727d1aefbc...,0.9803921568627451 1162.5751825494115 1678.675...
0,dbe222818fd1b10d0317e8b86bc8b1fa3ffd82b704cd52...,0.9882352941176471 2187.755092163466 979.63038...
0,4c462942f62aa048e5cc8c2a9480d426a293cac49c1c17...,0.9254901960784314 1107.2706886472488 1696.865...


In [15]:
df_pred.tail()

,Id,PredictionString
0,ea1508f6aa0a04e4325594d747c94196dfb9828e31f47b...,1.0 1437.878271147699 1473.224774396787 -21.44...
0,baa4ab6de7d367b19cadbf8d55d2210a6f4feaa392f02f...,0.996078431372549 2176.999155102856 986.302440...
0,95b3f3a457d298757da08d656e30a5206b8fe08b31dfdc...,0.9372549019607843 1379.3531168203617 1520.363...
0,d27829b2e2992502bc5ddc0eaf6546fa426c9b48eaa1ea...,0.9058823529411765 1138.0250901556644 1979.397...
0,fb270377ec483ac3cd4a56b40e179ee33ecb499d719a9e...,1.0 598.7128520980239 3381.267097051894 -5.513...


In [16]:
df_pred.to_csv('train_correct.csv',index=False)

In [17]:
def load_groundtruth_boxes(nuscenes, sample_tokens):
    gt_box3ds = []

    # Load annotations and filter predictions and annotations.
    for sample_token in tqdm(sample_tokens):

        sample = nuscenes.get('sample', sample_token)
        sample_annotation_tokens = sample['anns']
        
        for sample_annotation_token in sample_annotation_tokens:
            sample_annotation = nuscenes.get('sample_annotation', sample_annotation_token)
            sample_annotation_translation = sample_annotation['translation']
            
            class_name = sample_annotation['category_name']
            
            box3d = Box3D(
                sample_token=sample_token,
                translation=sample_annotation_translation,
                size=sample_annotation['size'],
                rotation=sample_annotation['rotation'],
                name=class_name
            )
            gt_box3ds.append(box3d)
            
    return gt_box3ds

gt_box3ds = load_groundtruth_boxes(level5data, sample_tokens)

In [18]:
len(gt_box3ds)

638179

In [19]:
pred_box3ds = []
pred_box3ds_correct = []
for i, sample_token in tqdm(enumerate(sample_tokens)):
    row = [] 
    token = sample_token
    df = train.loc[train['Id'] == token]
    pred_boxes = str(np.array(df['PredictionString'])[0])
    pred_boxes_list_3d = get_boxes_list_3d(pred_boxes)
    for j, pred_box in enumerate(pred_boxes_list_3d):
        cls_conf, x, y, z, w, l, h, yaw, cls_pred = pred_box
        cls_conf = float(cls_conf)
        x = float(x)
        y = float(y)
        z = float(z)
        w = float(w)
        h = float(h)
        l = float(l)
        yaw = float(yaw)
        q = euler2quat(0, 0, yaw)
        box3d = Box3D(
                    sample_token=token,
                    translation=[x, y, z],
                    size=[w, l, h],
                    rotation=[q[0],q[1],q[2],q[3]],
                    name=cls_pred,
                    score=cls_conf
                    )
        pred_box3ds.append(box3d)
    pred_boxes_correct = df_pred.iloc[i]['PredictionString']
    pred_boxes_list_3d_correct = get_boxes_list_3d(pred_boxes_correct)  
    for j, pred_box in enumerate(pred_boxes_list_3d_correct):
        cls_conf, x, y, z, w, l, h, yaw, cls_pred = pred_box
        cls_conf = float(cls_conf)
        x = float(x)
        y = float(y)
        z = float(z)
        w = float(w)
        h = float(h)
        l = float(l)
        yaw = float(yaw)
        q = euler2quat(0, 0, yaw)
        box3d_correct = Box3D(
                    sample_token=token,
                    translation=[x, y, z],
                    size=[w, l, h],
                    rotation=[q[0],q[1],q[2],q[3]],
                    name=cls_pred,
                    score=cls_conf
                    )
        pred_box3ds_correct.append(box3d_correct)

In [20]:
print(len(pred_box3ds), len(pred_box3ds_correct))

559171 559465


In [21]:
print(pred_box3ds[0], pred_box3ds_correct[0])

{'sample_token': '088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8d08a17f9b92b784184', 'translation': [1111.7313500943123, 1710.079884903943, -23.665222132632774], 'size': [2.4999038368109114, 5.499464661250448, 1.72], 'rotation': [0.8380014437949895, 0.0, 0.0, 0.5456680128040428], 'name': 'car', 'volume': 23.646788428153545, 'score': 1.0} {'sample_token': '088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8d08a17f9b92b784184', 'translation': [1111.7313500943123, 1710.079884903943, -23.665222132632774], 'size': [2.4999038368109114, 5.499464661250448, 1.72], 'rotation': [0.8380014437949895, 0.0, 0.0, 0.5456680128040428], 'name': 'car', 'volume': 23.646788428153545, 'score': 1.0}


In [22]:
import json
ARTIFACTS_FOLDER = "./artifacts"
os.makedirs(ARTIFACTS_FOLDER, exist_ok=True)
gt = [b.serialize() for b in gt_box3ds[:10000]]
pr = [b.serialize() for b in pred_box3ds[:10000]]

In [23]:
gt[0], pr[0]

({'sample_token': '088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8d08a17f9b92b784184',
  'translation': [1079.0989849262157, 1631.9507816978753, -23.759291955063574],
  'size': [1.943, 4.326, 1.393],
  'rotation': [-0.5997295321339267, 0, 0, 0.8002027794793152],
  'name': 'car',
  'volume': 11.708747273999998,
  'score': -1},
 {'sample_token': '088bcfd49c5599bb776cd76ef3a70e4ba0f31a94ef99a8d08a17f9b92b784184',
  'translation': [1111.7313500943123, 1710.079884903943, -23.665222132632774],
  'size': [2.4999038368109114, 5.499464661250448, 1.72],
  'rotation': [0.8380014437949895, 0.0, 0.0, 0.5456680128040428],
  'name': 'car',
  'volume': 23.646788428153545,
  'score': 1.0})

In [24]:
iou_th_range = np.linspace(0.5, 0.95, 10)
metric = {}
processes = []
output_dir = 'tmp/'
output_dir = Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

In [25]:
class_names = ['animal', 'bicycle', 'bus', 'car', 'emergency_vehicle',
                    'motorcycle', 'other_vehicle', 'pedestrian', 'truck']

In [26]:
def save_AP(gt, predictions, class_names, iou_threshold, output_dir):
    #computes average precisions (AP) for a given threshold, and saves the metrics in a temp file 
    # use lyft's provided function to compute AP
    AP = get_average_precisions(gt, predictions, class_names, iou_threshold)
    # create a dict with keys as class names and values as their respective APs
    metric = {c:AP[idx] for idx, c in enumerate(class_names)}

    # save the dict in a temp file
    summary_path = str(output_dir) + f'metric_summary_{iou_threshold}.json'
    with open(str(summary_path), 'w') as f:
        json.dump(metric, f)

In [27]:
def get_metric_overall_AP(iou_th_range, output_dir, class_names):
    ''' reads temp files and calculates overall per class APs.
    returns:
        `metric`: a dict with key as iou thresholds and value as dicts of class and their respective APs,
        `overall_AP`: overall AP of each class
    '''

    metric = {}
    overall_AP = np.zeros(len(class_names))
    for iou_threshold in iou_th_range:
        summary_path = str(output_dir) + f'metric_summary_{iou_threshold}.json'
        with open(str(summary_path), 'r') as f:
            data = json.load(f) # type(data): dict
            metric[iou_threshold] = data
            overall_AP += np.array([data[c] for c in class_names])
    overall_AP /= len(iou_th_range)
    return metric, overall_AP

In [28]:
time_start = time.time()
for iou_threshold in iou_th_range:
    process = Process(target=save_AP, args=(gt, pr, class_names, iou_threshold, output_dir))
    process.start()
    processes.append(process)

for process in processes:
    process.join()
print("Time to evaluate = ", time.time() - time_start)      

Time to evaluate =  135.18471789360046


In [29]:
# get overall metrics
metric, overall_AP = get_metric_overall_AP(iou_th_range, output_dir, class_names)
metric['overall'] = {c: overall_AP[idx] for idx, c in enumerate(class_names)}
metric['mAP'] = np.mean(overall_AP)
for th in iou_th_range:
    print("IOU threshold = ", th)
    average_precisions = list(metric[th].values())
    mAP = np.mean(average_precisions)
    print("Average per class mean average precision = ", mAP)
    for class_id in sorted(list(zip(class_names, average_precisions))):
        print(class_id)
    print("_______________________________________________")
print("Overall mean average precision = ", metric['mAP'])

IOU threshold =  0.5
Average per class mean average precision =  0.18412477796575658
('animal', 0.0)
('bicycle', 0.00024975024975024975)
('bus', 0.373999504852446)
('car', 0.3981798918561609)
('emergency_vehicle', 0.16666666666666666)
('motorcycle', 0.0)
('other_vehicle', 0.42875406839671976)
('pedestrian', 0.002517162471395881)
('truck', 0.28675595719867)
_______________________________________________
IOU threshold =  0.55
Average per class mean average precision =  0.1417077358684773
('animal', 0.0)
('bicycle', 0.00024975024975024975)
('bus', 0.30842539305284405)
('car', 0.26465201212068845)
('emergency_vehicle', 0.16666666666666666)
('motorcycle', 0.0)
('other_vehicle', 0.35777581422155924)
('pedestrian', 0.0)
('truck', 0.17759998650478676)
_______________________________________________
IOU threshold =  0.6
Average per class mean average precision =  0.07852327448699152
('animal', 0.0)
('bicycle', 0.00024975024975024975)
('bus', 0.24840671402110498)
('car', 0.14413321666860812)
('

In [30]:
time_start = time.time()
average_precisions = get_average_precisions(gt, pr, class_names, 0.01)
mAP = np.mean(average_precisions)
print("Average per class mean average precision = ", mAP)
for class_id in sorted(list(zip(class_names, average_precisions.flatten().tolist()))):
    print(class_id)
print("Time to evaluate = ", time.time() - time_start)  

Average per class mean average precision =  0.4766718502734284
('animal', 0.0)
('bicycle', 0.24543834235885315)
('bus', 0.5146182101941587)
('car', 0.7856778357275924)
('emergency_vehicle', 0.6666666666666666)
('motorcycle', 0.6476190476190476)
('other_vehicle', 0.6687575833229519)
('pedestrian', 0.1509083499046069)
('truck', 0.6103606166669775)
Time to evaluate =  32.019203186035156


In [31]:
pr_corr = [b.serialize() for b in pred_box3ds_correct[:10000]]

In [32]:
time_start = time.time()
for iou_threshold in iou_th_range:
    process = Process(target=save_AP, args=(gt, pr_corr, class_names, iou_threshold, output_dir))
    process.start()
    processes.append(process)

for process in processes:
    process.join()
print("Time to evaluate = ", time.time() - time_start)      

Time to evaluate =  125.5411205291748


In [33]:
# get overall metrics
metric, overall_AP = get_metric_overall_AP(iou_th_range, output_dir, class_names)
metric['overall'] = {c: overall_AP[idx] for idx, c in enumerate(class_names)}
metric['mAP'] = np.mean(overall_AP)
for th in iou_th_range:
    print("IOU threshold = ", th)
    average_precisions = list(metric[th].values())
    mAP = np.mean(average_precisions)
    print("Average per class mean average precision = ", mAP)
    for class_id in sorted(list(zip(class_names, average_precisions))):
        print(class_id)
    print("_______________________________________________")
print("Overall mean average precision = ", metric['mAP'])

IOU threshold =  0.5
Average per class mean average precision =  0.18431513280111908
('animal', 0.0010504201680672268)
('bicycle', 0.00024975024975024975)
('bus', 0.373999504852446)
('car', 0.3984007728642045)
('emergency_vehicle', 0.16666666666666666)
('motorcycle', 0.0)
('other_vehicle', 0.42904895624411477)
('pedestrian', 0.002517162471395881)
('truck', 0.2869029616934262)
_______________________________________________
IOU threshold =  0.55
Average per class mean average precision =  0.14186753129876192
('animal', 0.0010504201680672268)
('bicycle', 0.00024975024975024975)
('bus', 0.30842539305284405)
('car', 0.2647780529353794)
('emergency_vehicle', 0.16666666666666666)
('motorcycle', 0.0)
('other_vehicle', 0.3579620488084352)
('pedestrian', 0.0)
('truck', 0.17767544980771446)
_______________________________________________
IOU threshold =  0.6
Average per class mean average precision =  0.07854822815530839
('animal', 0.0)
('bicycle', 0.00024975024975024975)
('bus', 0.2484067140211

In [34]:
time_start = time.time()
average_precisions = get_average_precisions(gt, pr_corr, class_names, 0.01)
mAP = np.mean(average_precisions)
print("Average per class mean average precision = ", mAP)
for class_id in sorted(list(zip(class_names, average_precisions.flatten().tolist()))):
    print(class_id)
print("Time to evaluate = ", time.time() - time_start)  

Average per class mean average precision =  0.5512151403871266
('animal', 0.6675758069185121)
('bicycle', 0.245531982233051)
('bus', 0.5146424463639057)
('car', 0.7865244620837841)
('emergency_vehicle', 0.6666666666666666)
('motorcycle', 0.6476190476190476)
('other_vehicle', 0.6695520553498076)
('pedestrian', 0.1515267735715107)
('truck', 0.6112970226778545)
Time to evaluate =  30.580796003341675
